In [14]:
import pandas as pd
import numpy as np
import sklearn
import datetime
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow.parquet as pq

from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate
from surprise import Reader, Dataset
from surprise import KNNBaseline
from surprise.model_selection import GridSearchCV

from surprise import SVD, BaselineOnly, CoClustering

from tqdm import tqdm
import pickle
import itertools
from surprise import dump

In [2]:
train_group = pd.read_pickle('train_group_add_cookie.pickle')

In [3]:
user_id=train_group.index.get_level_values(0)
vacancy_id_=train_group.index.get_level_values(1)
data = pd.DataFrame(list(zip( user_id, vacancy_id_, train_group['event_type'].to_list())), columns = ['user_id', 'vacancy_id_', 'rating'])

data

,user_id,vacancy_id_,rating
0,000089d26cdd49d68839c68bc10f2cf2,177380,1
1,000089d26cdd49d68839c68bc10f2cf2,181353,1
2,000089d26cdd49d68839c68bc10f2cf2,195890,1
3,000089d26cdd49d68839c68bc10f2cf2,205562,1
4,000089d26cdd49d68839c68bc10f2cf2,211064,1
...,...,...,...
4662685,ffffdb17f45b4032b386d691d52e6c00,107125,10
4662686,ffffdb17f45b4032b386d691d52e6c00,107893,10
4662687,ffffdb17f45b4032b386d691d52e6c00,150843,10
4662688,ffffdb17f45b4032b386d691d52e6c00,160164,10


In [4]:
vacancy_ids = data['vacancy_id_']
  
#Подсчет количества упоминаний вакансий:
vacancy_ids_count=np.unique (np.asarray(vacancy_ids), return_counts= True )

print('Максимальное значение упоминаний вакансий: ', vacancy_ids_count[1].max())
print('Среднее значение упоминаний вакансий: ', np.average(vacancy_ids_count[1]))
print('Минимальное значение упоминаний вакансий: ', vacancy_ids_count[1].min())
print('Количество уникальных вакансий: ', len(vacancy_ids_count[1]))



Максимальное значение упоминаний вакансий:  18522
Среднее значение упоминаний вакансий:  29.111427447601567
Минимальное значение упоминаний вакансий:  1
Количество уникальных вакансий:  160167


In [5]:
df=data.groupby('user_id')['vacancy_id_']. nunique ()

data=data.loc[data['user_id'].isin(list(df[(df>=5) & (df<5000)].index))]

In [6]:
vacancy_ids = data['vacancy_id_']
  
#Подсчет количества упоминаний вакансий:
vacancy_ids_count=np.unique (np.asarray(vacancy_ids), return_counts= True )

print('Максимальное значение упоминаний вакансий: ', vacancy_ids_count[1].max())
print('Среднее значение упоминаний вакансий: ', np.average(vacancy_ids_count[1]))
print('Минимальное значение упоминаний вакансий: ', vacancy_ids_count[1].min())
print('Количество уникальных вакансий: ', len(vacancy_ids_count[1]))

Максимальное значение упоминаний вакансий:  18497
Среднее значение упоминаний вакансий:  29.098250521347136
Минимальное значение упоминаний вакансий:  1
Количество уникальных вакансий:  160162


In [7]:
reader = Reader(rating_scale=(0, 10))
data_ = Dataset.load_from_df(data[['user_id','vacancy_id_', 'rating']], reader)

In [10]:
print('Using ALS')
bsl_options = {'method': 'als', 
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)
cross_validate(algo, data_, measures=['RMSE', 'MAE'], cv=5, verbose=False)

Using ALS
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([2.89645861, 2.89822109, 2.89776584, 2.90305428, 2.89271569]),
 'test_mae': array([1.95999666, 1.96174128, 1.96074667, 1.96446073, 1.95885255]),
 'fit_time': (12.256006717681885,
  15.482001781463623,
  15.172055006027222,
  14.808998823165894,
  15.07599925994873),
 'test_time': (9.3660249710083,
  8.67099928855896,
  9.714027404785156,
  8.07599949836731,
  10.087000131607056)}

In [11]:
param_gridBO = {'bsl_options': {'method': ['als', 'sgd'],
        'reg': [1, 2],
        'learning_rate': [0.01, 0.05, 0.0025],
        'n_epochs': [5, 10, 15]}
        }
gsBO = GridSearchCV(BaselineOnly, param_gridBO, measures=['rmse', 'mae'], cv=3)


gsBO.fit(data_)


print('Best RMSE:', gsBO.best_score['rmse'], gsBO.best_params['rmse'])
print('Best MAE:', gsBO.best_score['mae'], gsBO.best_params['mae'])

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimati

In [19]:
bsl_options = {'method': 'als', 
               'learning_rate': 0.01,
               'n_epochs': 15,
               'reg_u': 12,
               'reg_i': 5
               }

trainset, testset = train_test_split(data_, test_size=0.25)
algo = BaselineOnly(bsl_options=bsl_options)
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 2.9010


2.9009843045116765

In [20]:
accuracy.mae(predictions)

MAE:  1.9661


1.9661130691244404

In [21]:
trainset = data_.build_full_trainset()
algo.fit(trainset)

Estimating biases using als...


In [22]:
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 2.7152


2.715205934879582

In [23]:
accuracy.mae(predictions)

MAE:  1.8269


1.8269302159729768

In [24]:
# save the model to disk
dump.dump('./dump_file', predictions, algo)

In [25]:
#Load the model from disk
predictions, algo = dump.load('./dump_file')

Проверка на тест данных

In [26]:
test_public_mfti=pd.read_pickle('test_public_mfti_add_all_vacansy_saw_same_user.pickle')
test_public_mfti

,cookie_id,vacancy_id_,user_id,list_vacansy_user_saw,all_vacansy_saw_same_user
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812...",477e719bfbcc4071a8acc1b357492b00,"{189576, 156688, 114834, 104082, 213013, 13442...","{131073, 131079, 131081, 131085, 131087, 13108..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348...",753a2a268ae84ab49475e62736c02860,"{240258, 176398, 195726, 116494, 139539, 21698...","{100001, 100002, 100003, 100006, 100007, 10000..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065...",d01c76b282364fbb8195f326de3e893c,"{195716, 141447, 191628, 206350, 117520, 25652...","{100001, 100002, 100003, 100006, 100007, 10000..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]",6adb4495090b454395d5aa2f1a5ff9d9,"{187652, 248331, 203404, 184726, 180382, 15824...","{100001, 100002, 100003, 100005, 100006, 10000..."
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]",b107696e3ff54e8a83182828bb845bc6,"{167168, 148864, 254087, 153353, 101389, 11726...","{131073, 131079, 131081, 131082, 131088, 13108..."
...,...,...,...,...,...
767,fdbcda17f22f406486837059e76c7fed,"[207851, 254989, 213344, 214180, 222146]",da74810d481449278a75a091178f9831,"{229314, 212971, 103567, 170333, 243294}","{196609, 196611, 229382, 196626, 196628, 22940..."
768,fe6193ab26494ace9be5aae36e507618,"[115352, 230546, 225527, 120188, 109360, 23212...",4da39c1e048d49cabcd1229fdcc6413f,"{181381, 140678, 108295, 198407, 253194, 21287...","{131074, 131085, 131088, 131122, 131135, 13114..."
769,fe95b2826ee1452b81201ed3f4c3294d,"[240362, 114852, 253946, 251081, 127546, 244688]",5207e9855ec54e02aa8cfa48ede76e1b,"{219841, 119075, 208350, 244646, 109990, 19575...","{131073, 131074, 131077, 131078, 131079, 13108..."
770,ff1aef256a49481698bb2e938510ff36,"[231194, 236363, 220747, 244688, 100094, 24052...",7f2067ab30994440b13a0ea37a5e8001,"{102794, 117134, 217615, 236821, 130587, 23403...","{131073, 131074, 131081, 131085, 131088, 13108..."


In [27]:
def generate_recommendation(model, user_id_, vacancy_ids_to_pred, n_items):


    test_set = [[user_id_, vacancy_id, 0] for vacancy_id in vacancy_ids_to_pred]



    #предсказание
    predictions = model.test(test_set)


    #топ n лучшие вакансии на основе предсказанных рейтингов
    pred_ratings = np.array([pred.est for pred in predictions])

    
    index_max = (-pred_ratings).argsort()[:n_items]

    return vacancy_ids_to_pred[index_max]

In [31]:
list_5=[]
list_10=[]
for i in tqdm(range(len(test_public_mfti))):
    user_id=test_public_mfti.user_id[i]
    vacancy_ids_to_pred = np.array(list(set(vacancy_ids_count[0]) & 
                                        (test_public_mfti.all_vacansy_saw_same_user[i]-test_public_mfti.list_vacansy_user_saw[i])))
    
    n_items=5
    list_5.append(generate_recommendation(algo,user_id,vacancy_ids_to_pred,n_items))
   
    n_items=10
    list_10.append(generate_recommendation(algo,user_id,vacancy_ids_to_pred,n_items))

100%|██████████| 772/772 [15:33<00:00,  1.21s/it]


In [32]:
test_public_mfti['5']=list_5
test_public_mfti['10']=list_10

test_public_mfti['inter_5']=[len(set(i) & set(j)) for i,j in zip(test_public_mfti['5'], test_public_mfti.vacancy_id_)]
test_public_mfti['inter_10']=[len(set(i) & set(j)) for i,j in zip(test_public_mfti['10'], test_public_mfti.vacancy_id_)]

In [33]:
sum(test_public_mfti['inter_5'])

0

In [34]:
sum(test_public_mfti['inter_10'])

4